# Predicciones con RandomForest

A continuación voy a aplicar mi modelo para predecir el tipo de sonido en base a un dataset nuevo de sonidos.

Empiezo importando las librerías que voy a necesitar.

In [12]:
import numpy as np
import pandas as pd
import pandas.io

import matplotlib.pyplot as plt

import librosa.display
import librosa

from glob import glob

import ffmpeg

import os

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.neighbors import KNeighborsClassifier

import pickle

Empiezo importando el modelo que he guardado anteriormente, para trabajar con él.

In [8]:
filename = 'finalized_model.sav'

loaded_model = pickle.load(open(filename, 'rb'))

Continúo descomponiendo los audios en segmentos de 10 segundos, y sacando todas sus features.

In [ ]:
def decompose_files(data_dir, audio_files):
    for j in range(len(audio_files)):
        myaudio = AudioSegment.from_file(data_dir + '{}'.format(os.listdir(data_dir)[j])) 
        chunk_length_ms = 10000
        chunks = make_chunks(myaudio, chunk_length_ms)

        for i, chunk in enumerate(chunks):
            chunk_name = "{}{}.wav".format(os.listdir(data_dir)[j], i)
            #print("exporting", chunk_name)
            chunk.export('./audios/predicciones_base/{}'.format(chunk_name), format="mp3")
            
decompose_files('./audios/predicciones/', glob('./audios/predicciones' + '/*'))

In [ ]:
def get_features(data_dir, audio_files):
    momentos = {'ducha': 'Ducha', 'cena': 'Cena', 'washing': 'Lavadora', 'vacuum': 'Aspiradora', 
                'shaver': 'Afeitadora', 'hairdryer': 'Secador_pelo', 'airconditioner': 'Aire_acondicionado', 
                'cellphone': 'Telefono', 'comp': 'Tecleo', 'silence': 'Silencio', 'dryer': 'Secadora', 
                'blender': 'Licuadora', 'doorbell': 'Timbre', 'alarm': 'Alarma', 'faucet': 'Grifo', 
                'microwave': 'Microondas'}
    features = []
    for i in range(len(audio_files)):
        y, sr = librosa.load(audio_files[i])
        name = os.listdir(data_dir)[i]
        pattern = "[._][\w]+"
        name = re.sub(pattern, '', name)
        if name in momentos.keys():
            momento = momentos.get(name)
        else:
            momento = 'Otro'
        mfcc = np.ndarray.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20))
        scem = np.ndarray.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
        scom = np.ndarray.mean(librosa.feature.spectral_contrast(S=np.abs(librosa.stft(y)), sr=sr))
        srom = np.ndarray.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
        sbwm = np.ndarray.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
        tempo = librosa.beat.tempo(onset_envelope=librosa.onset.onset_strength(y=y, sr=sr, hop_length=512), sr=sr, hop_length=512)[0]
        rmse = np.ndarray.mean(librosa.feature.rms(y=y))
        features.append([mfcc, scem, scom, srom, sbwm, tempo, rmse, momento])
    return pd.DataFrame(features, columns=['mfcc', 'scem','scom', 'srom','sbwm', 'tempo', 'rmse', 'momento'])

prueba = get_features('./audios/predicciones_base/', glob('./audios/predicciones_base' + '/*.wav'))

In [ ]:
prueba.to_csv('data_prueba.csv', index=False)

In [15]:
prueba = pd.read_csv('data_prueba.csv')
prueba.head()

,mfcc,scem,scom,srom,sbwm,tempo,rmse,momento
0,-1.606872,4092.840687,19.242309,7751.678339,3074.835521,129.199219,0.156137,Aspiradora
1,5.749998,673.601633,17.140124,1070.265145,1533.607014,129.199219,0.175553,Aire_acondicionado
2,-56.568542,0.000000,0.000000,0.000000,0.000000,120.000000,0.000000,Telefono
3,-12.618124,1806.021749,19.141861,4382.006836,2623.351950,120.000000,0.002939,Tecleo
4,-12.631319,4496.352321,16.700788,8196.106665,2977.760301,112.347147,0.029217,Tecleo


En este paso intermedio, dado que hay una pequeña cantidad de audios cuyas features son todas cero, los elimino para no ensuciar el modelo.

In [10]:
prueba = prueba[prueba['scem'] != 0].reset_index(drop=True)
prueba.head()

,mfcc,scem,scom,srom,sbwm,tempo,rmse,momento
0,-1.606872,4092.840687,19.242309,7751.678339,3074.835521,129.199219,0.156137,Aspiradora
1,5.749998,673.601633,17.140124,1070.265145,1533.607014,129.199219,0.175553,Aire_acondicionado
2,-12.618124,1806.021749,19.141861,4382.006836,2623.351950,120.000000,0.002939,Tecleo
3,-12.631319,4496.352321,16.700788,8196.106665,2977.760301,112.347147,0.029217,Tecleo
4,-18.866808,1595.430623,21.938086,2863.191581,1947.429463,184.570312,0.025959,Timbre


In [11]:
classes = ['Afeitadora', 'Aire_acondicionado', 'Alarma', 'Aspiradora', 'Cena', 'Ducha', 'Grifo', 'Lavadora', 
           'Licuadora', 'Microondas', 'Secador_pelo', 'Secadora', 'Silencio', 'Tecleo', 'Telefono', 'Timbre']

X_new = prueba.loc[:, prueba.columns != 'momento']
y_new = loaded_model.predict(X_new)
y_new2 = loaded_model.predict_proba(X_new)

reconocidos = 0
aciertos = 0
for i in range(len(X_new)):
    predicted_list = y_new[i].tolist()
    predicted_class = classes[y_new2[i].tolist().index(max(y_new2[i]))]
    predicted_probability = y_new2[i].tolist()[y_new2[i].tolist().index(max(y_new2[i]))]
    if max(y_new2[i]) > 0:
        if predicted_class == prueba.momento[i]:
            #print(str(i) + '-  {}: {:.2f}% ----> {}'.format(predicted_class, predicted_probability*100, prueba.momento[i]))
            aciertos += 1
            reconocidos += 1
        else:
            #print(str(i) + '-  {}: {:.2f}% ----> {}'.format(predicted_class, predicted_probability*100, prueba.momento[i]))
            reconocidos += 1


print('Porcentaje de sonidos reconocidos: {:.2f}%.'.format(reconocidos/(len(X_new))*100))
print('Porcentaje de aciertos: {:.2f}%.'.format(aciertos/(len(X_new))*100))
print('Porcentaje de aciertos una vez reconoce el sonido: {:.2f}%.'.format(aciertos/reconocidos*100))

Porcentaje de sonidos reconocidos: 100.00%.
Porcentaje de aciertos: 5.95%.
Porcentaje de aciertos una vez reconoce el sonido: 5.95%.


# Conclusiones

Como podemos observar, el modelo predice muy bien para sonidos que ya tiene dentro de la base de datos, pero es muy malo prediciendo sonidos nuevos. Esto probablemente se deba al overfitting, al tener un modelo muy ajustado para la propia base de datos sobre la que entrena.

Es por ello que voy a sacar más features características (transformada de Fourier) de cada clase, para conseguir un modelo con mejor capacidad de predicción.